## Load Data

In [1]:
pip install tensorflow

     -------------------------------------- 375.7/375.7 MB 6.8 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 92.3 MB/s eta 0:00:00
     ---------------------------------------- 135.6/135.6 KB ? eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
     ---------------------------------------- 434.6/434.6 KB ? eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 95.2 MB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 90.9 MB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
     --------------------------------------- 15.9/15.9 MB 93.9 MB/s eta 0:00:00
     ---------------------------------------- 209.4/209.4 KB ? eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 116.9 MB/s eta 0:00:

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Owner\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from os import listdir
from os.path import isfile, join
import tensorflow as tf
import numpy as np

# load neural responses. It is a np array with shape (200, 5) which corresponds
# to 200 images and 5 neurons.
neural_response_train = np.load("./neural_responses_train.npy")


# Code below is for loading and preprocessing images.
# Images will be loaded in train_iterator and test_iterator, which are two lists.
# elements in the list are np arrays with shape (10,224,224,3). 10 is batch size.
# Images are 224x224 with 3 color channels.
train_path = "./images/train"
# test_path = "./images/test"

batch_size = 10
img_height = 224
img_width = 224

trainfiles = [join(train_path, f) for f in listdir(train_path) if isfile(join(train_path, f))]
trainfiles = sorted(trainfiles)

# testfiles = [join(test_path, f) for f in listdir(test_path) if isfile(join(test_path, f))]
# testfiles = sorted(testfiles)

def load_image(im_file):
  image = tf.io.decode_jpeg(tf.io.read_file(im_file), channels=3)
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [img_height, img_width])
  image = tf.keras.applications.vgg16.preprocess_input(image)
  return image

train_dataset = tf.data.Dataset.from_tensor_slices(trainfiles)
train_dataset = train_dataset.map(load_image)
train_dataset = train_dataset.batch(10)
train_iterator = list(train_dataset.as_numpy_iterator())

# test_dataset = tf.data.Dataset.from_tensor_slices(testfiles)
# test_dataset = test_dataset.map(load_image)
# test_dataset = test_dataset.batch(10)
# test_iterator = list(test_dataset.as_numpy_iterator())

FileNotFoundError: [Errno 2] No such file or directory: './neural_responses_train.npy'

## Big Boi GridSearch (set up a shrine and pray)

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import Ridge, ElasticNet, BayesianRidge, Lasso, SGDRegressor, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from scipy.stats import pearsonr
import time
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel

# Define a more robust Pearson correlation coefficient scorer
def pearson_scorer(y_true, y_pred):
    try:
        # Check for constant values
        if np.all(y_true == y_true[0]) or np.all(y_pred == y_pred[0]):
            return 0.0  # Return 0 for constant predictions/targets
            
        # Check for NaN/Inf values
        if np.any(np.isnan(y_true)) or np.any(np.isnan(y_pred)) or \
           np.any(np.isinf(y_true)) or np.any(np.isinf(y_pred)):
            return 0.0  # Return 0 for NaN/Inf values
            
        # Calculate Pearson correlation
        r, _ = pearsonr(y_true, y_pred)
        
        # Handle NaN or Inf in the result
        if np.isnan(r) or np.isinf(r):
            return 0.0
            
        return r
    except Exception:
        # Return 0 for any other error
        return 0.0

# Create the scorer
pearson_scorer_sklearn = make_scorer(pearson_scorer, greater_is_better=True)

def analyze_neuron(train_iterator, neural_response_train, neuron_idx=0, 
                  cnn_models=None, max_layers=3, regression_models=None):
    """
    Find the best CNN, layer, and regression model for a specific neuron.
    
    Parameters:
    -----------
    train_iterator : Iterator
        Iterator that yields batches of images
    neural_response_train : numpy.ndarray
        Array of neural responses with shape (n_samples, n_neurons)
    neuron_idx : int, default=0
        Index of the neuron to train the model for (0-based)
    cnn_models : list or None, default=None
        List of CNN models to test: 'vgg16', 'resnet', 'efficientnet', 'inception'
    max_layers : int, default=3
        Maximum number of layers to test per CNN architecture
    regression_models : list or None, default=None
        List of regression models to test
        
    Returns:
    --------
    dict
        Dictionary containing the results for the best CNN, layer and model
    """
    start_time = time.time()
    
    # Define CNN models to test
    if cnn_models is None:
        cnn_models = ['vgg16', 'resnet', 'efficientnet', 'inception']

    # Define regression models to test (big boi list)
    if regression_models is None:
        regression_models = [
            'ridge', 'elasticnet', 'lasso', 'bayesianridge', 
            'randomforest', 'sgd', 'huber',
            'knn', 'adaboost', 'extratrees', 'gpr', 'linearsvr'
        ]
    
    print(f"Finding best model for Neuron {neuron_idx+1}")
    print(f"CNN models to test: {cnn_models}")
    print(f"Regression models to test: {regression_models}")
    
    # Store results for each CNN architecture
    all_cnn_results = []
    
    # Test each CNN architecture
    for cnn_type in cnn_models:
        best_layer_result = find_best_layer(
            train_iterator=train_iterator,
            neural_response_train=neural_response_train,
            neuron_idx=neuron_idx,
            model_type=cnn_type,
            max_layers=max_layers,
            regression_models=regression_models
        )
        if best_layer_result:
            all_cnn_results.append(best_layer_result)
    
    # Compare results from all CNN architectures
    if all_cnn_results:
        print("\nComparing best results from all CNN architectures:")
        print(f"{'CNN Type':^12}|{'Best Layer':^22}|{'Regression Model':^18}|{'Test r':^10}|{'p-value':^10}")
        
        for res in all_cnn_results:
            print(f"{res['cnn_type'].upper():^12}|{res['layer_name']:^22}|{res['regression_model_type']:^18}|{res['test_pearson_r']:.4f}|{res['test_pearson_p']:.4g}")
        
        # Select the best overall CNN and layer
        all_cnn_results.sort(key=lambda x: x['test_pearson_r'], reverse=True)
        best_result = all_cnn_results[0]
        
        print(f"\n{best_result['cnn_type'].upper()} performs best for Neuron {neuron_idx+1}")
        print(f"Best layer: {best_result['layer_name']}")
        print(f"Best regression model: {best_result['regression_model_type']}")
        print(f"Test Pearson's r: {best_result['test_pearson_r']:.4f} (p={best_result['test_pearson_p']:.4g})")
        
        # Add neuron index to the results
        best_result['neuron'] = neuron_idx + 1
        
        end_time = time.time()
        runtime = end_time - start_time
        print(f"Runtime: {runtime:.2f} seconds ({runtime/60:.2f} minutes)")
        
        return best_result
    else:
        print("No valid results found.")
        return None


def find_best_layer(train_iterator, neural_response_train, neuron_idx=0, 
                    model_type='vgg16', max_layers=3, regression_models=None):
    """
    Find the best CNN layer for predicting a specific neuron's responses.
    
    Parameters:
    -----------
    train_iterator : Iterator
        Iterator that yields batches of images
    neural_response_train : numpy.ndarray
        Array of neural responses with shape (n_samples, n_neurons)
    neuron_idx : int, default=0
        Index of the neuron to train the model for (0-based)
    model_type : str, default='vgg16'
        Type of CNN model to use ('vgg16', 'resnet', 'efficientnet', or 'inception')
    max_layers : int, default=3
        Maximum number of layers to test
    regression_models : list or None, default=None
        List of regression models to test
    
    Returns:
    --------
    dict or None
        Dictionary containing the results for the best layer and model, or None if no valid results
    """
    # Define layers to test based on selected model
    # Define layers to test based on selected model
    if model_type.lower() == 'vgg16':
        # All convolutional and pooling layers in VGG16
        all_cnn_layers = [
            # Block 1
            'block1_conv1', 'block1_conv2', 'block1_pool',
            # Block 2
            'block2_conv1', 'block2_conv2', 'block2_pool',
            # Block 3
            'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool',
            # Block 4
            'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool',
            # Block 5
            'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool'
        ]
        # Initialize VGG16 model
        base_model = VGG16(weights='imagenet', include_top=False)
        print(f"Initialized VGG16 model for feature extraction")
        
    elif model_type.lower() == 'resnet':
        # Key layers in ResNet50
        all_cnn_layers = [
            # Stage 1
            'conv1_relu', 'pool1_pool',
            # Stage 2
            'conv2_block1_out', 'conv2_block2_out', 'conv2_block3_out',
            # Stage 3
            'conv3_block1_out', 'conv3_block2_out', 'conv3_block3_out', 'conv3_block4_out',
            # Stage 4
            'conv4_block1_out', 'conv4_block2_out', 'conv4_block3_out', 
            'conv4_block4_out', 'conv4_block5_out', 'conv4_block6_out',
            # Stage 5
            'conv5_block1_out', 'conv5_block2_out', 'conv5_block3_out'
        ]
        # Initialize ResNet50 model
        base_model = ResNet50(weights='imagenet', include_top=False)
        print(f"Initialized ResNet50 model for feature extraction")
        
    elif model_type.lower() == 'efficientnet':
        # Key layers in EfficientNetB0
        all_cnn_layers = [
            'block1a_project_bn', 'block2a_project_bn', 'block2b_project_bn',
            'block3a_project_bn', 'block3b_project_bn', 'block4a_project_bn',
            'block4b_project_bn', 'block4c_project_bn', 'block5a_project_bn',
            'block5b_project_bn', 'block5c_project_bn', 'block6a_project_bn',
            'block6b_project_bn', 'block6c_project_bn', 'block6d_project_bn',
            'block7a_project_bn', 'top_activation'
        ]
        # Initialize EfficientNetB0 model
        base_model = EfficientNetB0(weights='imagenet', include_top=False)
        print(f"Initialized EfficientNetB0 model for feature extraction")
    
    elif model_type.lower() == 'inception':
        # Key layers in InceptionV3
        all_cnn_layers = [
            'activation', 'activation_1', 'activation_2', 'activation_3',
            'activation_4', 'max_pooling2d', 'activation_5', 'activation_6',
            'activation_7', 'activation_8', 'max_pooling2d_1', 'mixed0',
            'mixed1', 'mixed2', 'mixed3', 'mixed4', 'mixed5',
            'mixed6', 'mixed7', 'mixed8', 'mixed9', 'mixed10'
        ]
        # Initialize InceptionV3 model
        base_model = InceptionV3(weights='imagenet', include_top=False)
        print(f"Initialized InceptionV3 model for feature extraction")
        
    else:
        raise ValueError("model_type must be 'vgg16', 'resnet', 'efficientnet', or 'inception'")
    
    # Limit the number of layers to test if max_layers is less than total layers
    if max_layers < len(all_cnn_layers):
        selected_indices = np.linspace(0, len(all_cnn_layers) - 1, max_layers, dtype=int)
        layers_to_test = [all_cnn_layers[i] for i in selected_indices]
    else:
        layers_to_test = all_cnn_layers
    
    print(f"Testing layers: {layers_to_test}")
    
    # Store results for each layer
    all_layer_results = []
    
    # Test each layer
    for layer_name in layers_to_test:
        print(f"Evaluating layer: {layer_name}")
        
        try:
            # Create feature extraction model with the current layer
            feature_model = tf.keras.Model(inputs=base_model.input, 
                                          outputs=base_model.get_layer(layer_name).output)
            
            # Extract features from all batches
            all_features = []
            for batch in train_iterator:
                # Extract features
                features = feature_model(batch, training=False)
                # Global average pooling
                pooled = tf.keras.layers.GlobalAveragePooling2D()(features)
                all_features.append(pooled.numpy())
            
            features = np.concatenate(all_features, axis=0)
            # print(f"Features shape: {features.shape}")
            
            # Feature diagnostics
            # print(f"Feature statistics - min: {np.min(features)}, max: {np.max(features)}")
            # print(f"Any NaN in features: {np.any(np.isnan(features))}")
            # print(f"Any Inf in features: {np.any(np.isinf(features))}")
            
            # Split data into train and test sets
            X_train, X_test, y_train, y_test = train_test_split(
                features, neural_response_train, 
                test_size=0.2, random_state=42
            )
            
            # Get target for the selected neuron
            y_train_neuron = y_train[:, neuron_idx]
            y_test_neuron = y_test[:, neuron_idx]
            
            # Target diagnostics
            # print(f"Target statistics - min: {np.min(y_train_neuron)}, max: {np.max(y_train_neuron)}")
            # print(f"Target variance: {np.var(y_train_neuron)}")
            # print(f"Any NaN in target: {np.any(np.isnan(y_train_neuron))}")
            
            # Define cross-validation strategy - using 5-fold
            cv = KFold(n_splits=5, shuffle=True, random_state=42)
            
            # Define optimized model configurations
            model_configs = {
                'ridge': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', Ridge(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
                        'model__fit_intercept': [True, False],
                        'model__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag']
                    }
                },
                'lasso': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', Lasso(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
                        'model__fit_intercept': [True, False],
                        'model__selection': ['cyclic', 'random'],
                        'model__max_iter': [1000, 3000, 5000]
                    }
                },
                'elasticnet': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', ElasticNet(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
                        'model__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
                        'model__fit_intercept': [True, False],
                        'model__selection': ['cyclic', 'random'],
                        'model__max_iter': [1000, 3000, 5000]
                    }
                },
                'bayesianridge': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', BayesianRidge())
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__alpha_1': [1e-7, 1e-6, 1e-5, 1e-4],
                        'model__alpha_2': [1e-7, 1e-6, 1e-5, 1e-4],
                        'model__lambda_1': [1e-7, 1e-6, 1e-5, 1e-4],
                        'model__lambda_2': [1e-7, 1e-6, 1e-5, 1e-4],
                        'model__fit_intercept': [True, False],
                        'model__alpha_init': [None, 1.0, 10.0]
                    }
                },
                'randomforest': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', RandomForestRegressor(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__n_estimators': [50, 100, 200],
                        'model__max_depth': [3, 5, 8, 12, None],
                        'model__min_samples_leaf': [1, 3, 5, 10],
                        'model__min_samples_split': [2, 5, 10],
                        'model__max_features': ['sqrt', 'log2']
                    }
                },
                'extratrees': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', ExtraTreesRegressor(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__n_estimators': [50, 100, 200],
                        'model__max_depth': [3, 5, 8, None],
                        'model__min_samples_leaf': [1, 3, 5],
                        'model__max_features': ['sqrt', 'log2']
                    }
                },
                'adaboost': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', AdaBoostRegressor(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__n_estimators': [50, 100, 200],
                        'model__learning_rate': [0.01, 0.1, 1.0],
                        'model__loss': ['linear', 'square', 'exponential']
                    }
                },
                'linearsvr': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', LinearSVR(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__C': [0.01, 0.1, 1.0, 10.0, 100.0],
                        'model__epsilon': [0.0, 0.01, 0.1, 0.2],
                        'model__loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'],
                        'model__max_iter': [1000, 2000, 5000]
                    }
                },
                'sgd': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', SGDRegressor(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
                        'model__penalty': ['l2', 'l1', 'elasticnet'],
                        'model__alpha': [0.0001, 0.001, 0.01, 0.1],
                        'model__learning_rate': ['constant', 'optimal', 'adaptive'],
                        'model__eta0': [0.01, 0.1, 0.5]
                    }
                },
                'huber': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', HuberRegressor())
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__epsilon': [1.1, 1.35, 1.5, 2.0],
                        'model__alpha': [0.0001, 0.001, 0.01, 0.1],
                        'model__fit_intercept': [True, False],
                        'model__max_iter': [100, 500, 1000]
                    }
                },
                'knn': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', KNeighborsRegressor())
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 200, 'all'],
                        'model__n_neighbors': [3, 5, 7, 9, 15],
                        'model__weights': ['uniform', 'distance'],
                        'model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                        'model__leaf_size': [10, 30, 50]
                    }
                },
                'gpr': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', GaussianProcessRegressor(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [50, 100, 'all'],
                        'model__kernel': [
                            RBF(), 
                            RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0), 
                            DotProduct() + WhiteKernel(noise_level=1.0)
                        ],
                        'model__alpha': [1e-10, 1e-8, 1e-6],
                        'model__n_restarts_optimizer': [0, 1, 3]
                    }
                }
            }
            
            # Filter model configs based on requested regression models
            filtered_configs = {name: config for name, config in model_configs.items() 
                               if name in regression_models}
            
            # Run grid search for each selected model
            grid_results = {}
            
            for model_name, config in filtered_configs.items():
                print(f"  Testing {model_name.capitalize()}...")
                
                grid = GridSearchCV(
                    config['pipeline'],
                    config['param_grid'],
                    cv=cv,
                    scoring=pearson_scorer_sklearn,  # Use Pearson's r
                    n_jobs=-1,
                    verbose=0
                )
                
                try:
                    grid.fit(X_train, y_train_neuron)
                    
                    grid_results[model_name] = {
                        'grid': grid,
                        'best_score': grid.best_score_,
                        'best_params': grid.best_params_
                    }
                    
                    print(f"  {model_name.capitalize()} CV Pearson's r: {grid.best_score_:.4f}")
                except Exception as e:
                    print(f"  Error training {model_name.capitalize()}: {str(e)}")
                    continue
            
            # Find the best model overall
            if grid_results:
                best_model_name = max(grid_results, key=lambda k: grid_results[k]['best_score'])
                best_model = grid_results[best_model_name]['grid'].best_estimator_
                regression_model_type = best_model_name.capitalize()
                best_params = grid_results[best_model_name]['best_params']
                best_cv_score = grid_results[best_model_name]['best_score']
                
                # Evaluate on test set with the best model
                y_train_pred = best_model.predict(X_train)
                train_r, train_p = pearsonr(y_train_neuron, y_train_pred)
                train_rmse = np.sqrt(mean_squared_error(y_train_neuron, y_train_pred))
                
                y_test_pred = best_model.predict(X_test)
                test_r, test_p = pearsonr(y_test_neuron, y_test_pred)
                test_rmse = np.sqrt(mean_squared_error(y_test_neuron, y_test_pred))
                
                # Print diagnostic info about predictions
                print(f"  Train predictions - min: {np.min(y_train_pred)}, max: {np.max(y_train_pred)}")
                print(f"  Test predictions - min: {np.min(y_test_pred)}, max: {np.max(y_test_pred)}")
                print(f"  Any NaN in predictions: {np.any(np.isnan(y_train_pred)) or np.any(np.isnan(y_test_pred))}")
                
                print(f"  Best model: {regression_model_type}")
                print(f"  Best scaler: {best_model.named_steps['scaler'].__class__.__name__}")
                print(f"  Train: r = {train_r:.4f} (p={train_p:.4g}), RMSE = {train_rmse:.4f}")
                print(f"  Test: r = {test_r:.4f} (p={test_p:.4g}), RMSE = {test_rmse:.4f}")
                
                # Store results for this layer
                layer_results = {
                    'cnn_type': model_type,
                    'layer_name': layer_name,
                    'regression_model_type': regression_model_type,
                    'best_params': best_params,
                    'scaler_type': best_model.named_steps['scaler'].__class__.__name__,
                    'cv_pearson_r': best_cv_score,
                    'train_pearson_r': train_r,
                    'train_pearson_p': train_p,
                    'test_pearson_r': test_r,
                    'test_pearson_p': test_p,
                    'train_rmse': train_rmse,
                    'test_rmse': test_rmse,
                    'model': best_model
                }
                
                all_layer_results.append(layer_results)
        
        except Exception as e:
            print(f"Error processing layer {layer_name}: {str(e)}")
            continue
    
    # Sort results based on test Pearson's r
    if all_layer_results:
        all_layer_results.sort(key=lambda x: x['test_pearson_r'], reverse=True)
        
        # Print summary of all layers
        print("\nSummary of Results:")
        print(f"{'Layer Name':^22}|{'Regression Model':^18}|{'CV r':^10}|{'Test r':^10}")
        
        for res in all_layer_results:
            print(f"{res['layer_name']:^22}|{res['regression_model_type']:^18}|{res['cv_pearson_r']:.4f}|{res['test_pearson_r']:.4f}")
        
        # Return the best layer result
        best_layer_result = all_layer_results[0]
        return best_layer_result
    else:
        return None

In [ ]:
result = analyze_neuron(
    neuron_idx=0,
    train_iterator=train_iterator,
    neural_response_train=neural_response_train,
    cnn_models=['vgg16', 'resnet', 'efficientnet', 'inception'],
    regression_models=  ['ridge', 'elasticnet', 'lasso', 'bayesianridge', 'randomforest', 'sgd', 'huber', 'knn', 'adaboost', 'extratrees', 'gpr', 'linearsvr'],
    max_layers=50
)

# Print quick summary
print('\n\n'+'='*10+'RESULT'+'='*10)
print(f"Best CNN: {result['cnn_type']}")
print(f"Best layer: {result['layer_name']}")
print(f"Best scaler: {result['scaler_type']}")
print(f"Best regression model: {result['regression_model_type']}")
print(f"Best params: {result['best_params']}")
print(f"Train Pearson's r: {result['train_pearson_r']:.4f}")
print(f"Test Pearson's r: {result['test_pearson_r']:.4f}")

Finding best model for Neuron 1
CNN models to test: ['vgg16']
Regression models to test: ['svr', 'sgd', 'huber', 'knn', 'gpr', 'linearsvr']
Initialized VGG16 model for feature extraction
Testing layers: ['block1_conv1']
Evaluating layer: block1_conv1
  Testing Linearsvr...


c:\Users\ethan\anaconda3\envs\nimbus\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ethan\anaconda3\envs\nimbus\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


  Linearsvr CV Pearson's r: 0.0784
  Testing Sgd...


c:\Users\ethan\anaconda3\envs\nimbus\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:776: UserWarning: k=100 is greater than n_features=64. All the features will be returned.
  warnings.warn(
c:\Users\ethan\anaconda3\envs\nimbus\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1575: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


  Sgd CV Pearson's r: 0.2502
  Testing Huber...


c:\Users\ethan\anaconda3\envs\nimbus\Lib\site-packages\sklearn\linear_model\_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


  Huber CV Pearson's r: 0.0581
  Testing Knn...


c:\Users\ethan\anaconda3\envs\nimbus\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:776: UserWarning: k=100 is greater than n_features=64. All the features will be returned.
  warnings.warn(


  Knn CV Pearson's r: 0.0904
  Testing Gpr...
  Gpr CV Pearson's r: 0.0892
  Train predictions - min: -5679.690546000719, max: 5435.543096577406
  Test predictions - min: -5183.314569438219, max: 5732.039190327406
  Any NaN in predictions: False
  Best model: Sgd
  Best scaler: RobustScaler
  Train: r = 0.0138 (p=0.8628), RMSE = 1870.0378
  Test: r = 0.1080 (p=0.5071), RMSE = 2091.7319

Summary of Results:
      Layer Name      | Regression Model |   CV r   |  Test r  
     block1_conv1     |       Sgd        |0.2502|0.1080

Comparing best results from all CNN architectures:
  CNN Type  |      Best Layer      | Regression Model |  Test r  | p-value  
   VGG16    |     block1_conv1     |       Sgd        |0.1080|0.5071

VGG16 performs best for Neuron 1
Best layer: block1_conv1
Best regression model: Sgd
Test Pearson's r: 0.1080 (p=0.5071)
Runtime: 62.95 seconds (1.05 minutes)


==========RESULT==========
Best CNN: vgg16
Best layer: block1_conv1
Best scaler: RobustScaler
Best regression 

## Smaller boi

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import Ridge, ElasticNet, BayesianRidge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from scipy.stats import pearsonr
import time

# Define a more robust Pearson correlation coefficient scorer
def pearson_scorer(y_true, y_pred):
    try:
        # Check for constant values
        if np.all(y_true == y_true[0]) or np.all(y_pred == y_pred[0]):
            return 0.0  # Return 0 for constant predictions/targets
            
        # Check for NaN/Inf values
        if np.any(np.isnan(y_true)) or np.any(np.isnan(y_pred)) or \
           np.any(np.isinf(y_true)) or np.any(np.isinf(y_pred)):
            return 0.0  # Return 0 for NaN/Inf values
            
        # Calculate Pearson correlation
        r, _ = pearsonr(y_true, y_pred)
        
        # Handle NaN or Inf in the result
        if np.isnan(r) or np.isinf(r):
            return 0.0
            
        return r
    except Exception:
        # Return 0 for any other error
        return 0.0

# Create the scorer
pearson_scorer_sklearn = make_scorer(pearson_scorer, greater_is_better=True)

def analyze_neuron(train_iterator, neural_response_train, neuron_idx=0, 
                  cnn_models=None, max_layers=3, regression_models=None):
    """
    Find the best CNN, layer, and regression model for a specific neuron.
    
    Parameters:
    -----------
    train_iterator : Iterator
        Iterator that yields batches of images
    neural_response_train : numpy.ndarray
        Array of neural responses with shape (n_samples, n_neurons)
    neuron_idx : int, default=0
        Index of the neuron to train the model for (0-based)
    cnn_models : list or None, default=None
        List of CNN models to test: 'vgg16', 'resnet', 'efficientnet', 'inception'
    max_layers : int, default=3
        Maximum number of layers to test per CNN architecture
    regression_models : list or None, default=None
        List of regression models to test
        
    Returns:
    --------
    dict
        Dictionary containing the results for the best CNN, layer and model
    """
    start_time = time.time()
    
    # Define CNN models to test
    if cnn_models is None:
        cnn_models = ['vgg16', 'resnet', 'efficientnet', 'inception']

    # Define regression models to test (focused list for speed)
    if regression_models is None:
        regression_models = ['ridge', 'elasticnet', 'lasso', 'bayesianridge', 'randomforest', 'gbr', 'svr', 'linearsvr']
    
    print(f"Finding best model for Neuron {neuron_idx+1}")
    print(f"CNN models to test: {cnn_models}")
    print(f"Regression models to test: {regression_models}")
    
    # Store results for each CNN architecture
    all_cnn_results = []
    
    # Test each CNN architecture
    for cnn_type in cnn_models:
        best_layer_result = find_best_layer(
            train_iterator=train_iterator,
            neural_response_train=neural_response_train,
            neuron_idx=neuron_idx,
            model_type=cnn_type,
            max_layers=max_layers,
            regression_models=regression_models
        )
        if best_layer_result:
            all_cnn_results.append(best_layer_result)
    
    # Compare results from all CNN architectures
    if all_cnn_results:
        print("\nComparing best results from all CNN architectures:")
        print(f"{'CNN Type':^12}|{'Best Layer':^22}|{'Regression Model':^18}|{'Test r':^10}|{'p-value':^10}")
        
        for res in all_cnn_results:
            print(f"{res['cnn_type'].upper():^12}|{res['layer_name']:^22}|{res['regression_model_type']:^18}|{res['test_pearson_r']:.4f}|{res['test_pearson_p']:.4g}")
        
        # Select the best overall CNN and layer
        all_cnn_results.sort(key=lambda x: x['test_pearson_r'], reverse=True)
        best_result = all_cnn_results[0]
        
        print(f"\n{best_result['cnn_type'].upper()} performs best for Neuron {neuron_idx+1}")
        print(f"Best layer: {best_result['layer_name']}")
        print(f"Best regression model: {best_result['regression_model_type']}")
        print(f"Test Pearson's r: {best_result['test_pearson_r']:.4f} (p={best_result['test_pearson_p']:.4g})")
        
        # Add neuron index to the results
        best_result['neuron'] = neuron_idx + 1
        
        end_time = time.time()
        runtime = end_time - start_time
        print(f"Runtime: {runtime:.2f} seconds ({runtime/60:.2f} minutes)")
        
        return best_result
    else:
        print("No valid results found.")
        return None


def find_best_layer(train_iterator, neural_response_train, neuron_idx=0, 
                    model_type='vgg16', max_layers=3, regression_models=None):
    """
    Find the best CNN layer for predicting a specific neuron's responses.
    
    Parameters:
    -----------
    train_iterator : Iterator
        Iterator that yields batches of images
    neural_response_train : numpy.ndarray
        Array of neural responses with shape (n_samples, n_neurons)
    neuron_idx : int, default=0
        Index of the neuron to train the model for (0-based)
    model_type : str, default='vgg16'
        Type of CNN model to use ('vgg16', 'resnet', 'efficientnet', or 'inception')
    max_layers : int, default=3
        Maximum number of layers to test
    regression_models : list or None, default=None
        List of regression models to test
    
    Returns:
    --------
    dict or None
        Dictionary containing the results for the best layer and model, or None if no valid results
    """
    # Define layers to test based on selected model
    if model_type.lower() == 'vgg16':
        all_cnn_layers = [
            'block1_pool', 'block2_pool', 'block3_pool', 'block4_pool', 'block5_pool'
        ]
        base_model = VGG16(weights='imagenet', include_top=False)
        print(f"Testing VGG16 model")
        
    elif model_type.lower() == 'resnet':
        all_cnn_layers = [
            'conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 
            'conv4_block6_out', 'conv5_block3_out'
        ]
        base_model = ResNet50(weights='imagenet', include_top=False)
        print(f"Testing ResNet50 model")
        
    elif model_type.lower() == 'efficientnet':
        all_cnn_layers = [
            'block2b_add', 'block3b_add', 'block4c_add',
            'block5c_add', 'block6d_add', 'top_activation'
        ]
        base_model = EfficientNetB0(weights='imagenet', include_top=False)
        print(f"Testing EfficientNetB0 model")
    
    elif model_type.lower() == 'inception':
        all_cnn_layers = [
            'max_pooling2d', 'max_pooling2d_1', 'mixed2', 
            'mixed5', 'mixed7', 'mixed10'
        ]
        base_model = InceptionV3(weights='imagenet', include_top=False)
        print(f"Testing InceptionV3 model")
        
    else:
        raise ValueError("model_type must be 'vgg16', 'resnet', 'efficientnet', or 'inception'")
    
    # Limit the number of layers to test if max_layers is less than total layers
    if max_layers < len(all_cnn_layers):
        selected_indices = np.linspace(0, len(all_cnn_layers) - 1, max_layers, dtype=int)
        layers_to_test = [all_cnn_layers[i] for i in selected_indices]
    else:
        layers_to_test = all_cnn_layers
    
    print(f"Testing layers: {layers_to_test}")
    
    # Store results for each layer
    all_layer_results = []
    
    # Test each layer
    for layer_name in layers_to_test:
        print(f"Evaluating layer: {layer_name}")
        
        try:
            # Create feature extraction model with the current layer
            feature_model = tf.keras.Model(inputs=base_model.input, 
                                          outputs=base_model.get_layer(layer_name).output)
            
            # Extract features from all batches
            all_features = []
            for batch in train_iterator:
                # Extract features
                features = feature_model(batch, training=False)
                # Global average pooling
                pooled = tf.keras.layers.GlobalAveragePooling2D()(features)
                all_features.append(pooled.numpy())
            
            features = np.concatenate(all_features, axis=0)
            # print(f"Features shape: {features.shape}")
            
            # Feature diagnostics
            # print(f"Feature statistics - min: {np.min(features)}, max: {np.max(features)}")
            # print(f"Any NaN in features: {np.any(np.isnan(features))}")
            # print(f"Any Inf in features: {np.any(np.isinf(features))}")
            
            # Split data into train and test sets
            X_train, X_test, y_train, y_test = train_test_split(
                features, neural_response_train, 
                test_size=0.2, random_state=42
            )
            
            # Get target for the selected neuron
            y_train_neuron = y_train[:, neuron_idx]
            y_test_neuron = y_test[:, neuron_idx]
            
            # Target diagnostics
            # print(f"Target statistics - min: {np.min(y_train_neuron)}, max: {np.max(y_train_neuron)}")
            # print(f"Target variance: {np.var(y_train_neuron)}")
            # print(f"Any NaN in target: {np.any(np.isnan(y_train_neuron))}")
            
            # Define cross-validation strategy - using 5-fold
            cv = KFold(n_splits=5, shuffle=True, random_state=42)
            
            # Define optimized model configurations
            model_configs = {
                'ridge': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', Ridge(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [100, 200, 'all'],
                        'model__alpha': [0.1, 1.0, 10.0]
                    }
                },
                'lasso': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', Lasso(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [100, 200, 'all'],
                        'model__alpha': [0.01, 0.1, 1.0]
                    }
                },
                'elasticnet': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', ElasticNet(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [100, 200, 'all'],
                        'model__alpha': [0.01, 0.1, 1.0],
                        'model__l1_ratio': [0.1, 0.5, 0.9]
                    }
                },
                'bayesianridge': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', BayesianRidge())
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [100, 200, 'all'],
                        'model__alpha_1': [1e-6, 1e-4],
                        'model__alpha_2': [1e-6, 1e-4]
                    }
                },
                'randomforest': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', RandomForestRegressor(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [100, 200, 'all'],
                        'model__n_estimators': [50, 100],
                        'model__max_depth': [5, 10]
                    }
                },
                'gbr': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', GradientBoostingRegressor(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [100, 200, 'all'],
                        'model__n_estimators': [50, 100],
                        'model__learning_rate': [0.01, 0.1]
                    }
                },
                'svr': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', SVR())
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [100, 200, 'all'],
                        'model__C': [0.1, 1.0, 10.0],
                        'model__kernel': ['linear', 'rbf']
                    }
                },
                'linearsvr': {
                    'pipeline': Pipeline([
                        ('scaler', None),  # Will be set by grid search
                        ('feature_selection', SelectKBest(f_regression)),
                        ('model', LinearSVR(random_state=42))
                    ]),
                    'param_grid': {
                        'scaler': [StandardScaler(), RobustScaler()],
                        'feature_selection__k': [100, 200, 'all'],
                        'model__C': [0.1, 1.0, 10.0]
                    }
                }
            }

            # Filter model configs based on requested regression models
            filtered_configs = {name: config for name, config in model_configs.items() 
                               if name in regression_models}
            
            # Run grid search for each selected model
            grid_results = {}
            
            for model_name, config in filtered_configs.items():
                print(f"  Testing {model_name.capitalize()}...")
                
                grid = GridSearchCV(
                    config['pipeline'],
                    config['param_grid'],
                    cv=cv,
                    scoring=pearson_scorer_sklearn,  # Use Pearson's r
                    n_jobs=-1,
                    verbose=0
                )
                
                try:
                    grid.fit(X_train, y_train_neuron)
                    
                    grid_results[model_name] = {
                        'grid': grid,
                        'best_score': grid.best_score_,
                        'best_params': grid.best_params_
                    }
                    
                    print(f"  {model_name.capitalize()} CV Pearson's r: {grid.best_score_:.4f}")
                except Exception as e:
                    print(f"  Error training {model_name.capitalize()}: {str(e)}")
                    continue
            
            # Find the best model overall
            if grid_results:
                best_model_name = max(grid_results, key=lambda k: grid_results[k]['best_score'])
                best_model = grid_results[best_model_name]['grid'].best_estimator_
                regression_model_type = best_model_name.capitalize()
                best_params = grid_results[best_model_name]['best_params']
                best_cv_score = grid_results[best_model_name]['best_score']
                
                # Evaluate on test set with the best model
                y_train_pred = best_model.predict(X_train)
                train_r, train_p = pearsonr(y_train_neuron, y_train_pred)
                train_rmse = np.sqrt(mean_squared_error(y_train_neuron, y_train_pred))
                
                y_test_pred = best_model.predict(X_test)
                test_r, test_p = pearsonr(y_test_neuron, y_test_pred)
                test_rmse = np.sqrt(mean_squared_error(y_test_neuron, y_test_pred))
                
                # Print diagnostic info about predictions
                print(f"  Train predictions - min: {np.min(y_train_pred)}, max: {np.max(y_train_pred)}")
                print(f"  Test predictions - min: {np.min(y_test_pred)}, max: {np.max(y_test_pred)}")
                print(f"  Any NaN in predictions: {np.any(np.isnan(y_train_pred)) or np.any(np.isnan(y_test_pred))}")
                
                print(f"  Best model: {regression_model_type}")
                print(f"  Best scaler: {best_model.named_steps['scaler'].__class__.__name__}")
                print(f"  Train: r = {train_r:.4f} (p={train_p:.4g}), RMSE = {train_rmse:.4f}")
                print(f"  Test: r = {test_r:.4f} (p={test_p:.4g}), RMSE = {test_rmse:.4f}")
                
                # Store results for this layer
                layer_results = {
                    'cnn_type': model_type,
                    'layer_name': layer_name,
                    'regression_model_type': regression_model_type,
                    'best_params': best_params,
                    'scaler_type': best_model.named_steps['scaler'].__class__.__name__,
                    'cv_pearson_r': best_cv_score,
                    'train_pearson_r': train_r,
                    'train_pearson_p': train_p,
                    'test_pearson_r': test_r,
                    'test_pearson_p': test_p,
                    'train_rmse': train_rmse,
                    'test_rmse': test_rmse,
                    'model': best_model
                }
                
                all_layer_results.append(layer_results)
        
        except Exception as e:
            print(f"Error processing layer {layer_name}: {str(e)}")
            continue
    
    # Sort results based on test Pearson's r
    if all_layer_results:
        all_layer_results.sort(key=lambda x: x['test_pearson_r'], reverse=True)
        
        # Print summary of all layers
        print("\nSummary of Results:")
        print(f"{'Layer Name':^22}|{'Regression Model':^18}|{'CV r':^10}|{'Test r':^10}")
        
        for res in all_layer_results:
            print(f"{res['layer_name']:^22}|{res['regression_model_type']:^18}|{res['cv_pearson_r']:.4f}|{res['test_pearson_r']:.4f}")
        
        # Return the best layer result
        best_layer_result = all_layer_results[0]
        return best_layer_result
    else:
        return None

In [ ]:
result = analyze_neuron(
    neuron_idx=0,
    train_iterator=train_iterator,
    neural_response_train=neural_response_train,
    cnn_models=['vgg16', 'resnet', 'efficientnet', 'inception'],
    regression_models=['ridge', 'elasticnet', 'lasso', 'bayesianridge', 'randomforest', 'gbr', 'svr', 'linearsvr'],
    max_layers=50
)

# Print quick summary
print('\n\n'+'='*10+'RESULT'+'='*10)
print(f"Best CNN: {result['cnn_type']}")
print(f"Best layer: {result['layer_name']}")
print(f"Best scaler: {result['scaler_type']}")
print(f"Best regression model: {result['regression_model_type']}")
print(f"Best params: {result['best_params']}")
print(f"Train Pearson's r: {result['train_pearson_r']:.4f}")
print(f"Test Pearson's r: {result['test_pearson_r']:.4f}")

Finding best model for Neuron 1
CNN models to test: ['vgg16']
Regression models to test: ['ridge']
Testing VGG16 model
Testing layers: ['block1_pool', 'block5_pool']
Evaluating layer: block1_pool
Features shape: (200, 64)
Feature statistics - min: 0.013451282866299152, max: 2302.58349609375
Any NaN in features: False
Any Inf in features: False
Target statistics - min: 0.0, max: 3.267901659011841
Target variance: 0.2589110732078552
Any NaN in target: False
  Testing Ridge...
  Ridge CV Pearson's r: 0.0674
  Train predictions - min: -0.23347827792167664, max: 0.8190208673477173
  Test predictions - min: -0.18321236968040466, max: 0.9697402715682983
  Any NaN in predictions: False
  Best model: Ridge
  Best scaler: StandardScaler
  Train: r = 0.5929 (p=1.466e-16), RMSE = 0.4184
  Test: r = 0.0184 (p=0.9105), RMSE = 0.8773
Evaluating layer: block5_pool


c:\Users\ethan\anaconda3\envs\nimbus\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:776: UserWarning: k=100 is greater than n_features=64. All the features will be returned.
  warnings.warn(


Features shape: (200, 512)
Feature statistics - min: 0.0, max: 143.68621826171875
Any NaN in features: False
Any Inf in features: False
Target statistics - min: 0.0, max: 3.267901659011841
Target variance: 0.2589110732078552
Any NaN in target: False
  Testing Ridge...
  Ridge CV Pearson's r: 0.3369
  Train predictions - min: -0.014868184924125671, max: 3.2236123085021973
  Test predictions - min: -0.4157315790653229, max: 2.239962100982666
  Any NaN in predictions: False
  Best model: Ridge
  Best scaler: RobustScaler
  Train: r = 0.9996 (p=2.666e-250), RMSE = 0.0169
  Test: r = 0.0250 (p=0.8785), RMSE = 0.9785

Summary of Results:
      Layer Name      | Regression Model |   CV r   |  Test r  
     block5_pool      |      Ridge       |0.3369|0.0250
     block1_pool      |      Ridge       |0.0674|0.0184

Comparing best results from all CNN architectures:
  CNN Type  |      Best Layer      | Regression Model |  Test r  | p-value  
   VGG16    |     block5_pool      |      Ridge       |